In [1]:
%env BASE_DIR=/home/jupyter

env: BASE_DIR=/home/jupyter


In [26]:
!gsutil cp gs://cs327e-open-access/shopify.zip $BASE_DIR

Copying gs://cs327e-open-access/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [6]:
!unzip $BASE_DIR/shopify.zip

Archive:  /home/jupyter/shopify.zip
   creating: shopify/
  inflating: __MACOSX/._shopify      
  inflating: shopify/apps_categories.csv  
  inflating: __MACOSX/shopify/._apps_categories.csv  
  inflating: shopify/categories.csv  
  inflating: __MACOSX/shopify/._categories.csv  
  inflating: shopify/reviews.csv     
  inflating: __MACOSX/shopify/._reviews.csv  
  inflating: shopify/.DS_Store       
  inflating: __MACOSX/shopify/._.DS_Store  
  inflating: shopify/key_benefits.csv  
  inflating: __MACOSX/shopify/._key_benefits.csv  
  inflating: shopify/pricing_plan_features.csv  
  inflating: __MACOSX/shopify/._pricing_plan_features.csv  
  inflating: shopify/pricing_plans.csv  
  inflating: __MACOSX/shopify/._pricing_plans.csv  
  inflating: shopify/apps.csv        
  inflating: __MACOSX/shopify/._apps.csv  


In [28]:
import pandas as pd
import uuid
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

count = 0
for row in rows:
    record = {}
    record['app_id'] = row[0]
    record['author'] = row[1]
    record['rating'] = row[2]
    record['posted_at'] = row[3]
    
    review_id = str(uuid.uuid4())
    review_ref = db.collection('reviews').document(review_id)
    batch.set(review_ref, record)
    count += 1
    if count > 400:
        count = 0
        batch.commit()
        batch = db.batch()

In [29]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
category_rows = df.values.tolist()

df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
app_category_rows = df.values.tolist()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

count = 0
for category_row in category_rows:
    record = {}
    record['id'] = category_row[0]
    record['title'] = category_row[1]
    
    category_ref = db.collection('categories').document(category_row[0])
    batch.set(category_ref, record)
    
    
    for app_category_row in app_category_rows:
        if category_row[0] == app_category_row[1]:
            ac_record = {}
            ac_record['app_id'] = app_category_row[0]
            ac_record['category_id'] = app_category_row[1]
            
            ac_id = str(uuid.uuid4())
            ac_ref = category_ref.collection('apps_categories').document(ac_id)
            
            batch.set(ac_ref, ac_record)
            
            for app_row in app_rows:
                if app_category_row[0] == app_row[0]:
                    
                    app_record = {}
                    app_record['app_id'] = app_row[0]
                    app_record['url'] = app_row[1]
                    app_record['title'] = app_row[2]
                    app_record['developer'] = app_row[3]
                    app_record['developer_link'] = app_row[4]
                    app_record['icon'] = app_row[5]
                    app_record['rating'] = app_row[6]
                    app_record['reviews_count'] = app_row[7]
            
                    app_ref = ac_ref.collection('apps').document(app_row[0])
                    batch.set(app_ref, app_record)
                    count += 1
                    if count > 400:
                        count = 0
                    batch.commit()
                    batch = db.batch()

In [30]:
import pandas as pd
import uuid
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
benefit_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pp_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()

for app_row in app_rows:
    app_record = {}
    app_record['id'] = app_row[0]
    app_record['url'] = app_row[1]
    app_record['title'] = app_row[2]
    app_record['developer'] = app_row[3]
    app_record['developer_link'] = app_row[4]
    app_record['icon'] = app_row[5]
    app_record['rating'] = app_row[6]
    app_record['reviews_count'] = app_row[7]
    
    app_ref = db.collection('apps').document(app_row[0])
    
    batch.set(app_ref, app_record)
    
    for benefit_row in benefit_rows:
        if app_row[0] == benefit_row[0]:
            benefit_record = {}
            benefit_record['title'] = benefit_row[1]
            benefit_record['description'] = benefit_row[2]
            
            benefit_id = str(uuid.uuid4())
            benefit_ref = app_ref.collection('key benefits').document(benefit_id)
            
            batch.set(benefit_ref, benefit_record)
            
    for pp_row in pp_rows:
        if app_row[0] == pp_row[1]:
            pp_record = {}
            pp_record['id'] = pp_row[0]
            pp_record['title'] = pp_row[2]
            pp_record['price'] = pp_row[3]
            
            pp_ref = app_ref.collection('pricing plans').document(pp_row[0])
            
            batch.set(pp_ref, pp_record)
            
            for ppf_row in ppf_rows:
                if pp_row[0] == ppf_row[1]:
                    ppf_record = {}
                    ppf_record['feature'] = ppf_row[2]
                    
                    ppf_id = str(uuid.uuid4())
                    ppf_ref = pp_ref.collection('features').document(ppf_id)
                    
                    batch.set(ppf_ref, ppf_record)
            
    batch.commit()
    
    

In [31]:
from google.cloud import firestore
db = firestore.Client()

apps_ref = db.collection('apps')
count = 0
benefit_count = 0
plan_count = 0
for results in apps_ref.stream():
    count += 1
    for ben in apps_ref.document(results.id).collection('key benefits').stream():
        benefit_count += 1
    for plan in apps_ref.document(results.id).collection('pricing plans').stream():
        plan_count += 1
print('There are ' + str(count) + ' documents in the apps collection')
print('There are ' + str(sub_count) + ' documents in the key benefits subcollection')
print('There are ' + str(plan_count) + ' documents in the pricing plans subcollection')

There are 3547 documents in the apps collection
There are 19202 documents in the key benefits subcollection
There are 6275 documents in the pricing plans subcollection


In [32]:
from google.cloud import firestore
db = firestore.Client()

cats_ref = db.collection('categories')
count = 0
appc_count = 0
for result in cats_ref.stream():
    count += 1
    for appcat in cats_ref.document(result.id).collection('apps_categories').stream():
        appc_count += 1
print('There are ' + str(count) + ' documents in the categories collection')
print('There are ' + str(appc_count) + ' documents in the app categories subcollection')

There are 13 documents in the categories collection
There are 16149 documents in the app categories subcollection


Top 10 Productivity Apps (whose categories.title = "Productivity") sorted
by their rating in descending order. Return the id, title, developer, rating and
reviews_count for those apps. Limit the results to the first 10 records.

In [52]:
from google.cloud import firestore
db = firestore.Client() 

cats_ref = db.collection('categories')
query = cats_ref.where('title', '==', 'Productivity').order_by(
    'rating', "DESCENDING").limit(10) 
results = query.stream()

for doc in results:
    print('Document: ' + str(doc.to_dict()))
    


In [44]:
from google.cloud import firestore 
db = firestore.Client()

app_ref = db.collection('apps')
query = app_ref.where('reviews_count', '>', 2000).order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

for doc in results:
    print('Document: ' + str(doc.to_dict()))

Document: {'developer_link': 'https://apps.shopify.com/partners/privy', 'url': 'https://apps.shopify.com/privy', 'icon': 'https://apps.shopifycdn.com/listing_images/f3d61c764a54b22aed51cd1cc31b5a74/icon/620368ed2c537a42eaa1dc82593622e3.png?height=84&width=84', 'title': 'Privy ‑ Exit Pop Ups & Email', 'reviews_count': 23078, 'id': 'd9f142ee-b141-4dc4-9353-173db61d2eb0', 'rating': 4.7, 'developer': 'Privy'}
Document: {'reviews_count': 8737, 'id': '78ea0810-c008-4a4e-a82f-de0c790e3286', 'url': 'https://apps.shopify.com/free-shipping-bar', 'developer': 'Hextom', 'developer_link': 'https://apps.shopify.com/partners/hextom', 'rating': 4.9, 'icon': 'https://apps.shopifycdn.com/listing_images/635cce572624d1086ac37403cf25b5ee/icon/390b8ec270438793fd37f05da12ee6bb.png?height=84&width=84', 'title': 'Free Shipping Bar'}
Document: {'reviews_count': 6905, 'rating': 4.8, 'developer_link': 'https://apps.shopify.com/partners/carlsapps', 'id': 'b88488b0-9912-44d3-b736-224c36f09d95', 'icon': 'https://app